# 🚗 AV Simulation on Google Colab - Enhanced Setup Guide

This notebook provides a comprehensive setup for the Autonomous Vehicle Simulation system on Google Colab with robust error handling and dynamic path detection.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aanshshah/av-simulation/blob/main/examples/notebooks/01_colab_setup.ipynb)

## 📋 What You'll Learn
- Automatic environment detection and setup
- Robust dependency installation with error handling
- Dynamic path configuration for any environment
- Virtual display setup with fallback options
- Memory-optimized simulation helpers

## ⚡ Prerequisites
- Google account for Colab access
- Basic Python knowledge
- Understanding of Jupyter notebooks

## Quick Start Flow

1. Run this setup notebook end-to-end.
2. Open **02_simulation_runner.ipynb** to generate fresh data (optional toggle available).
3. Open **03_data_analysis.ipynb** for plots – it falls back to bundled sample data if no run is present.
4. Use **04_visualization_examples.ipynb** and **05_advanced_analysis.ipynb** for deeper dives.


## 🔧 Step 1: Enhanced Environment Setup

This cell performs comprehensive dependency installation with error handling and verification.

In [ ]:
    # Enhanced dependency installation with curated requirements
    import sys
    import subprocess
    import os
    from pathlib import Path

    def install_dependencies():
        """Install required dependencies with minimal footprint."""
        print("🚀 Starting Enhanced Dependency Installation")
        print("=" * 50)

        is_colab = (
            'COLAB_GPU' in os.environ or
            'COLAB_TPU_ADDR' in os.environ or
            '/content' in os.getcwd()
        )

        print(f"🌍 Environment: {'Google Colab' if is_colab else 'Local/Other'}")
        print(f"📁 Working directory: {os.getcwd()}")

        requirements_url = os.environ.get(
            'AV_SIM_REQUIREMENTS_URL',
            'https://raw.githubusercontent.com/aanshshah/av-simulation/main/examples/colab_requirements.txt'
        )
        requirements_path = Path('/content/av-simulation-colab-requirements.txt' if is_colab else 'colab_requirements.txt')

        print("
🐍 Downloading curated requirements list...")
        try:
            if requirements_path.exists():
                requirements_path.unlink()
            result = subprocess.run(
                ['curl', '-sSfL', requirements_url, '-o', str(requirements_path)],
                capture_output=True, text=True, timeout=60
            )
            if result.returncode != 0:
                raise RuntimeError(result.stderr.strip())
            print(f"  ✅ Saved to {requirements_path}")
        except Exception as download_error:
            print(f"  ❌ Failed to fetch curated list: {download_error}")
            print("  🔁 Falling back to inline dependency list")
            requirements_path.write_text('
'.join([
                'pygame==2.5.2', 'pyvirtualdisplay>=3.0', 'pillow>=10.0',
                'numpy>=1.23', 'pandas>=1.5', 'matplotlib>=3.7',
                'seaborn>=0.12', 'plotly>=5.18', 'scipy>=1.10', 'scikit-learn>=1.2'
            ]))

        print("
📦 Installing Python packages (this takes ~1 minute in Colab)...")
        install_cmd = [sys.executable, '-m', 'pip', 'install', '--quiet', '-r', str(requirements_path)]
        try:
            result = subprocess.run(install_cmd, capture_output=True, text=True, timeout=600)
            if result.returncode == 0:
                print("  ✅ Dependencies installed from curated list")
            else:
                print(f"  ❌ Installation issues: {result.stderr[:200]}")
        except subprocess.TimeoutExpired:
            print("  ⏰ Installation timed out — consider rerunning this cell")

        print("
🔍 Verifying core imports...")
        test_imports = ['pygame', 'pyvirtualdisplay', 'PIL', 'pandas', 'matplotlib']
        for module in test_imports:
            try:
                __import__(module if module != 'PIL' else 'PIL.Image')
                print(f"  ✅ {module}")
            except Exception as verify_error:
                print(f"  ❌ {module}: {verify_error}")

        print("
✅ Dependency installation complete!")
        return True

    install_dependencies()


## 📥 Step 2: Project Setup and File Management

This cell handles project file setup with multiple options and dynamic path detection.

In [ ]:
    # Enhanced project setup with dynamic paths and bundle download
    import os
    import sys
    import subprocess
    import shutil
    from pathlib import Path

    def setup_project_files():
        """Retrieve project files quickly (bundle download preferred)."""
        print("📁 Project File Setup")
        print("=" * 30)

        is_colab = (
            'COLAB_GPU' in os.environ or
            '/content' in os.getcwd()
        )

        base_path = '/content' if is_colab else os.getcwd()
        project_path = os.path.join(base_path, 'av-simulation')
        src_path = os.path.join(project_path, 'src')
        data_dir = os.path.join(project_path, 'examples', 'notebooks', 'data')

        print(f"📍 Base path: {base_path}")
        print(f"📍 Project path: {project_path}")
        print(f"📍 Source path: {src_path}")

        if os.path.exists(src_path):
            print("✅ Simulation files already present")
            setup_python_path(src_path)
            return True

        print("
🔍 Simulation files not found. Options:")
        print("1. 📦 Download project bundle (recommended)")
        print("2. 📤 Upload files manually")
        print("3. 🏗️ Create lightweight development scaffold")

        bundle_url = os.environ.get(
            'AV_SIM_BUNDLE_URL',
            'https://github.com/aanshshah/av-simulation/archive/refs/heads/main.zip'
        )
        bundle_path = os.path.join(base_path, 'av-simulation-bundle.zip')

        print("
📦 Attempting to download project bundle...")
        try:
            if os.path.exists(bundle_path):
                os.remove(bundle_path)
            result = subprocess.run(
                ['curl', '-L', '-o', bundle_path, bundle_url],
                capture_output=True, text=True, timeout=180
            )
            if result.returncode != 0:
                raise RuntimeError(result.stderr.strip())

            extract_root = os.path.join(base_path, 'av-simulation-main')
            if os.path.exists(extract_root):
                shutil.rmtree(extract_root)

            subprocess.run(['unzip', '-q', bundle_path, '-d', base_path], check=True)

            extracted_paths = [
                p for p in Path(base_path).glob('av-simulation-*')
                if p.is_dir() and (p / 'src').exists()
            ]
            if extracted_paths:
                source_root = str(extracted_paths[0])
                if os.path.exists(project_path):
                    shutil.rmtree(project_path)
                shutil.move(source_root, project_path)
                print("  ✅ Bundle downloaded and extracted")
            else:
                raise RuntimeError('Expected src folder not found in bundle')
        except Exception as bundle_error:
            print(f"  ⚠️ Bundle download failed: {bundle_error}")
            print("  🔁 Falling back to scaffold creation")
            create_project_structure(project_path)

        setup_python_path(src_path)
        os.makedirs(data_dir, exist_ok=True)

        print("
💡 Next steps:")
        print("  - 02_simulation_runner.ipynb → run a short simulation")
        print("  - 03_data_analysis.ipynb → explore results (uses sample data if needed)")
        print("  - 04_visualization_examples.ipynb → additional plots")

        return True

    def create_project_structure(project_path):
        """Create minimal project layout for manual uploads."""
        directories = [
            'src/av_simulation',
            'src/av_simulation/core',
            'src/av_simulation/data',
            'src/av_simulation/detection',
            'src/av_simulation/planning',
            'examples/notebooks',
            'examples/notebooks/data',
            'examples/data',
            'examples/utils',
            'simulation_data'
        ]
        for directory in directories:
            full_path = os.path.join(project_path, directory)
            os.makedirs(full_path, exist_ok=True)

        init_files = [
            'src/av_simulation/__init__.py',
            'src/av_simulation/core/__init__.py',
            'src/av_simulation/data/__init__.py',
            'src/av_simulation/detection/__init__.py',
            'src/av_simulation/planning/__init__.py'
        ]
        for init_file in init_files:
            full_path = os.path.join(project_path, init_file)
            Path(full_path).write_text('# Package initialization
')
        print("  ✅ Lightweight structure ready")

    def setup_python_path(src_path):
        """Ensure source path is on sys.path."""
        if src_path not in sys.path:
            sys.path.insert(0, src_path)
            print(f"  ✅ Added to Python path: {src_path}")
        else:
            print(f"  ✅ Already on Python path: {src_path}")

    setup_project_files()


## 🖥️ Step 3: Robust Virtual Display Setup

This cell sets up virtual display with comprehensive error handling and fallback options.

In [ ]:
# Robust virtual display setup
import os
import time

def setup_virtual_display(width=1200, height=800, retries=3):
    """Setup virtual display with retry logic and fallbacks"""
    print("🖥️ Virtual Display Setup")
    print("=" * 25)
    
    # Check if we're in an environment that needs virtual display
    is_colab = (
        'COLAB_GPU' in os.environ or 
        '/content' in os.getcwd()
    )
    
    if not is_colab:
        print("ℹ️ Not in Colab environment, virtual display may not be needed")
    
    display = None
    
    for attempt in range(retries):
        print(f"\n🔄 Attempt {attempt + 1}/{retries}")
        
        try:
            from pyvirtualdisplay import Display
            
            # Stop any existing display
            if display:
                try:
                    display.stop()
                except:
                    pass
            
            # Create new display
            display = Display(visible=0, size=(width, height))
            display.start()
            
            # Set environment variable
            os.environ['DISPLAY'] = ':' + str(display.display)
            
            print(f"  ✅ Display created: {width}x{height}")
            print(f"  🔗 Display ID: {os.environ.get('DISPLAY')}")
            
            # Test the display
            if test_display():
                print("  ✅ Display test successful")
                return display
            else:
                print("  ❌ Display test failed")
                
        except ImportError:
            print("  ❌ pyvirtualdisplay not available")
            break
        except Exception as e:
            print(f"  ❌ Display setup failed: {e}")
        
        if attempt < retries - 1:
            print("  ⏳ Waiting before retry...")
            time.sleep(2)
    
    print("\n❌ Virtual display setup failed after all attempts")
    print("💡 You may need to:")
    print("  - Restart the runtime")
    print("  - Run the dependency installation cell again")
    print("  - Check system package installation")
    
    return None

def test_display():
    """Test if display is working with pygame"""
    try:
        import pygame
        pygame.init()
        screen = pygame.display.set_mode((100, 100))
        pygame.display.flip()
        pygame.quit()
        return True
    except Exception as e:
        print(f"    Display test error: {e}")
        return False

# Setup virtual display
virtual_display = setup_virtual_display()

if virtual_display:
    print("\n🎉 Virtual display ready for simulation!")
else:
    print("\n⚠️ Continuing without virtual display (may cause issues with pygame)")

## 🛠️ Step 4: Enhanced Colab Helper Setup

This cell creates enhanced helper functions with memory management and error handling.

In [ ]:
# Enhanced Colab helpers with memory management
import threading
import time
from typing import Dict, List, Optional, Any
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

class EnhancedColabSimulationRunner:
    """Enhanced simulation runner with memory management and error handling"""
    
    def __init__(self, max_screenshots: int = 50, max_memory_mb: int = 500):
        self.screenshots = []
        self.max_screenshots = max_screenshots
        self.max_memory_mb = max_memory_mb
        self.simulation_data = {}
        self.running = False
        
    def capture_screenshot(self, screen, metadata: Optional[Dict] = None) -> Dict:
        """Capture screenshot with memory management"""
        try:
            # Get screen data
            w, h = screen.get_size()
            raw = pygame.image.tostring(screen, 'RGB')
            image = Image.frombytes('RGB', (w, h), raw)
            
            # Create screenshot data
            screenshot_data = {
                'image': image,
                'timestamp': time.time(),
                'size': (w, h),
                'metadata': metadata or {}
            }
            
            # Memory management - remove old screenshots if limit reached
            if len(self.screenshots) >= self.max_screenshots:
                removed = self.screenshots.pop(0)
                print(f"🗑️ Removed old screenshot (keeping last {self.max_screenshots})")
            
            self.screenshots.append(screenshot_data)
            
            # Memory usage check
            estimated_mb = len(self.screenshots) * w * h * 3 / (1024 * 1024)
            if estimated_mb > self.max_memory_mb:
                print(f"⚠️ High memory usage: {estimated_mb:.1f}MB")
            
            return screenshot_data
            
        except Exception as e:
            print(f"❌ Screenshot capture failed: {e}")
            return {}
    
    def run_headless_simulation(self, config: Dict) -> Optional[Dict]:
        """Run simulation with enhanced error handling"""
        print(f"🚀 Starting Enhanced Simulation")
        print(f"⚙️ Config: {config}")
        
        # Validate configuration
        duration = config.get('duration', 30)
        screenshot_interval = config.get('screenshot_interval', 2.0)
        
        if duration > 300:  # 5 minutes max
            print("⚠️ Duration limited to 5 minutes for Colab compatibility")
            duration = 300
        
        # Try to import simulation
        try:
            from av_simulation.core.simulation import Simulation
            print("✅ Simulation module imported")
        except ImportError as e:
            print(f"❌ Cannot import simulation: {e}")
            print("💡 Make sure you've run the setup cells above")
            return None
        
        # Create simulation with error handling
        try:
            sim = Simulation(enable_data_collection=config.get('collect_data', True))
            print("✅ Simulation instance created")
        except Exception as e:
            print(f"❌ Failed to create simulation: {e}")
            return None
        
        # Run simulation loop
        start_time = time.time()
        last_screenshot = 0
        frame_count = 0
        
        try:
            import pygame
            
            while time.time() - start_time < duration:
                current_time = time.time() - start_time
                
                # Handle pygame events
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        print("🛑 Simulation stopped by quit event")
                        break
                
                # Update simulation
                try:
                    if hasattr(sim, 'current_env') and sim.current_env.ego_vehicle:
                        action = sim.planner.plan_action(sim.current_env.ego_vehicle)
                        sim.current_env.ego_vehicle.set_action(action)
                    
                    sim.current_env.step(1.0/60)  # 60 FPS
                    frame_count += 1
                    
                    # Collect data
                    if sim.data_collection_enabled:
                        sim.collect_simulation_data()
                        
                except Exception as e:
                    print(f"⚠️ Simulation step error: {e}")
                    break
                
                # Capture screenshots
                if current_time - last_screenshot >= screenshot_interval:
                    try:
                        sim.current_env.draw(sim.screen)
                        if hasattr(sim, 'draw_hud'):
                            sim.draw_hud()
                        pygame.display.flip()
                        
                        metadata = {
                            'simulation_time': current_time,
                            'frame_count': frame_count,
                            'fps': frame_count / current_time if current_time > 0 else 0
                        }
                        
                        self.capture_screenshot(sim.screen, metadata)
                        last_screenshot = current_time
                        
                        print(f"📸 Screenshot at t={current_time:.1f}s (frame {frame_count})")
                        
                    except Exception as e:
                        print(f"⚠️ Screenshot error: {e}")
                
                time.sleep(1/60)  # Maintain 60 FPS
            
            # Cleanup
            pygame.quit()
            
            # Save data
            run_id = None
            if sim.data_collection_enabled and hasattr(sim, 'current_run_id'):
                try:
                    sim.data_repository.end_current_run()
                    run_id = sim.current_run_id
                    print(f"💾 Data saved: {run_id}")
                except Exception as e:
                    print(f"⚠️ Data save error: {e}")
            
            # Return results
            results = {
                'run_id': run_id,
                'duration': time.time() - start_time,
                'screenshots': len(self.screenshots),
                'frames': frame_count,
                'avg_fps': frame_count / (time.time() - start_time)
            }
            
            print(f"\n✅ Simulation completed:")
            print(f"   Duration: {results['duration']:.1f}s")
            print(f"   Screenshots: {results['screenshots']}")
            print(f"   Average FPS: {results['avg_fps']:.1f}")
            
            return results
            
        except Exception as e:
            print(f"❌ Simulation error: {e}")
            pygame.quit()
            return None
    
    def display_screenshots(self, max_images: int = 6, figsize: tuple = (15, 10)):
        """Display screenshots with enhanced layout"""
        if not self.screenshots:
            print("📷 No screenshots to display")
            return
        
        n_images = min(len(self.screenshots), max_images)
        cols = 3
        rows = (n_images + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=figsize)
        if rows == 1:
            axes = [axes] if cols == 1 else axes
        else:
            axes = axes.flatten()
        
        for i in range(n_images):
            screenshot = self.screenshots[i]
            axes[i].imshow(screenshot['image'])
            
            # Enhanced title with metadata
            metadata = screenshot['metadata']
            title_parts = []
            
            if 'simulation_time' in metadata:
                title_parts.append(f"t={metadata['simulation_time']:.1f}s")
            if 'frame_count' in metadata:
                title_parts.append(f"frame={metadata['frame_count']}")
            if 'fps' in metadata:
                title_parts.append(f"fps={metadata['fps']:.1f}")
            
            title = " | ".join(title_parts) if title_parts else f"Screenshot {i+1}"
            axes[i].set_title(title, fontsize=10)
            axes[i].axis('off')
        
        # Hide empty subplots
        for i in range(n_images, len(axes)):
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"📊 Displayed {n_images} of {len(self.screenshots)} screenshots")
    
    def get_memory_usage(self) -> Dict[str, Any]:
        """Get memory usage information"""
        if not self.screenshots:
            return {'screenshots': 0, 'estimated_mb': 0}
        
        # Estimate memory usage
        sample_screenshot = self.screenshots[0]
        w, h = sample_screenshot['size']
        bytes_per_screenshot = w * h * 3  # RGB
        total_bytes = len(self.screenshots) * bytes_per_screenshot
        total_mb = total_bytes / (1024 * 1024)
        
        return {
            'screenshots': len(self.screenshots),
            'estimated_mb': round(total_mb, 2),
            'max_screenshots': self.max_screenshots,
            'avg_size': (w, h)
        }
    
    def clear_screenshots(self):
        """Clear all screenshots to free memory"""
        count = len(self.screenshots)
        self.screenshots.clear()
        print(f"🗑️ Cleared {count} screenshots")

# Create enhanced runner instance
enhanced_runner = EnhancedColabSimulationRunner()

print("✅ Enhanced Colab simulation runner created!")
print(f"📊 Configuration:")
print(f"   Max screenshots: {enhanced_runner.max_screenshots}")
print(f"   Max memory: {enhanced_runner.max_memory_mb}MB")

## 🎮 Step 5: Enhanced Display Test

Test pygame functionality with comprehensive error handling and diagnostics.

In [ ]:
# Enhanced pygame test with diagnostics
import pygame
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def comprehensive_display_test():
    """Comprehensive display and pygame test"""
    print("🎮 Comprehensive Display Test")
    print("=" * 30)
    
    # Environment check
    print(f"🌍 DISPLAY environment: {os.environ.get('DISPLAY', 'Not set')}")
    
    try:
        # Initialize pygame
        pygame.init()
        print("✅ Pygame initialized")
        
        # Get available drivers
        drivers = pygame.display.get_driver()
        print(f"🔧 Display driver: {drivers}")
        
        # Create display
        screen_size = (400, 300)
        screen = pygame.display.set_mode(screen_size)
        pygame.display.set_caption("AV Simulation Display Test")
        print(f"✅ Display created: {screen_size[0]}x{screen_size[1]}")
        
        # Draw test pattern
        print("🎨 Drawing test pattern...")
        
        # Background gradient
        for y in range(screen_size[1]):
            color_value = int(50 + (y / screen_size[1]) * 100)
            pygame.draw.line(screen, (color_value, color_value, color_value), (0, y), (screen_size[0], y))
        
        # Test shapes and colors
        pygame.draw.circle(screen, (0, 255, 0), (100, 150), 30)    # Green circle (ego vehicle)
        pygame.draw.circle(screen, (255, 0, 0), (300, 150), 25)    # Red circle (other vehicle)
        pygame.draw.rect(screen, (255, 255, 0), (50, 250, 300, 10)) # Yellow road marking
        pygame.draw.polygon(screen, (0, 100, 255), [(200, 50), (180, 100), (220, 100)])  # Blue triangle
        
        # Add some text-like rectangles (since font rendering might be complex)
        pygame.draw.rect(screen, (255, 255, 255), (20, 20, 100, 20))  # White rectangle as "text"
        
        pygame.display.flip()
        print("✅ Test pattern drawn")
        
        # Capture and convert to PIL Image
        w, h = screen.get_size()
        raw = pygame.image.tostring(screen, 'RGB')
        image = Image.frombytes('RGB', (w, h), raw)
        print("✅ Screenshot captured")
        
        # Test image properties
        img_array = np.array(image)
        print(f"📊 Image info:")
        print(f"   Size: {image.size}")
        print(f"   Mode: {image.mode}")
        print(f"   Array shape: {img_array.shape}")
        print(f"   Pixel range: {img_array.min()} - {img_array.max()}")
        print(f"   Mean color: {img_array.mean(axis=(0,1))}")
        
        # Cleanup pygame
        pygame.quit()
        print("✅ Pygame cleanup completed")
        
        # Display the captured image
        plt.figure(figsize=(10, 8))
        plt.imshow(image)
        plt.title("🎮 Pygame Display Test - Virtual Display Working!", fontsize=14, fontweight='bold')
        plt.axis('off')
        
        # Add some info text
        info_text = f"Display Size: {w}x{h}\nDriver: {drivers}\nDisplay: {os.environ.get('DISPLAY', 'Not set')}"
        plt.figtext(0.02, 0.02, info_text, fontsize=10, 
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8))
        
        plt.tight_layout()
        plt.show()
        
        print("\n🎉 Display test successful! Ready for AV simulation.")
        return True
        
    except Exception as e:
        print(f"❌ Display test failed: {e}")
        print("\n🔧 Troubleshooting suggestions:")
        print("  1. Restart the runtime and run all setup cells again")
        print("  2. Check that virtual display is properly set up")
        print("  3. Verify all dependencies are installed")
        print("  4. Try running the dependency installation cell again")
        
        try:
            pygame.quit()
        except:
            pass
        
        return False

# Run comprehensive test
test_success = comprehensive_display_test()

## 🔍 Step 6: System Verification and Import Testing

Verify that all components are working and simulation modules can be imported.

In [ ]:
    # Comprehensive system verification
    import sys
    import os
    from pathlib import Path

    def verify_complete_setup():
        """Verify that the complete setup is working."""
        print("🔍 Complete System Verification")
        print("=" * 35)

        verification_results = {}

        base_path = '/content' if ('/content' in os.getcwd() or 'COLAB_GPU' in os.environ) else os.getcwd()
        project_path = Path(base_path) / 'av-simulation'
        src_path = project_path / 'src'
        sample_data_path = project_path / 'examples' / 'notebooks' / 'data' / 'sample_ego_run.csv'

        # 1. Environment check
        print("
1. 🌍 Environment Check")
        is_colab = '/content' in base_path
        verification_results['environment'] = is_colab
        print(f"   {'✅' if is_colab else '⚠️'} Running in {'Colab' if is_colab else 'local'} environment")

        # 2. Dependency imports
        print("
2. 📦 Dependency Check")
        required_modules = ['pygame', 'pyvirtualdisplay', 'PIL', 'pandas', 'matplotlib']
        dependency_status = {}
        for module in required_modules:
            try:
                __import__(module if module != 'PIL' else 'PIL.Image')
                dependency_status[module] = True
            except Exception:
                dependency_status[module] = False
        for module, ok in dependency_status.items():
            print(f"   {'✅' if ok else '❌'} {module}")
        verification_results['dependencies'] = all(dependency_status.values())

        # 3. Display / pygame
        print("
3. 🖥️ Display Setup")
        display_env = os.environ.get('DISPLAY')
        has_display = bool(display_env)
        verification_results['display'] = has_display
        print(f"   {'✅' if has_display else '⚠️'} DISPLAY={display_env}")

        # 4. Project structure
        print("
4. 📁 Project Structure")
        structure_checks = {
            'project_exists': project_path.exists(),
            'src_exists': src_path.exists(),
            'utils_exists': (project_path / 'examples' / 'utils').exists(),
        }
        for name, ok in structure_checks.items():
            print(f"   {'✅' if ok else '❌'} {name.replace('_', ' ')}")
        verification_results['project_structure'] = all(structure_checks.values())

        # 5. Sample data availability
        print("
5. 🗂️ Sample Data")
        sample_ok = sample_data_path.exists()
        print(f"   {'✅' if sample_ok else '⚠️'} {sample_data_path}")
        verification_results['sample_data'] = sample_ok

        # 6. Simulation imports
        print("
6. 🧠 Simulation Imports")
        if verification_results['project_structure']:
            if str(src_path) not in sys.path:
                sys.path.insert(0, str(src_path))
            try:
                from av_simulation.core.simulation import Simulation  # noqa: F401
                from av_simulation.data.repository import DataRepository  # noqa: F401
                print("   ✅ Core modules available")
                verification_results['simulation_imports'] = True
            except Exception as import_error:
                print(f"   ❌ Import error: {import_error}")
                verification_results['simulation_imports'] = False
        else:
            print("   ⚠️ Skipping import check (project structure incomplete)")
            verification_results['simulation_imports'] = False

        total_checks = len(verification_results)
        passed_checks = sum(1 for ok in verification_results.values() if ok)

        print("
" + "=" * 35)
        print("📋 Verification Summary")
        print("=" * 35)
        for check, status in verification_results.items():
            icon = '✅' if status else '❌'
            print(f"   {icon} {check.replace('_', ' ').title()}")
        print(f"
📊 Overall: {passed_checks}/{total_checks} checks passed")

        if passed_checks == total_checks:
            print("
🎉 SETUP COMPLETE! All systems ready for simulation.")
            print("   → Next run 02_simulation_runner.ipynb")
        elif passed_checks >= total_checks * 0.7:
            print("
⚠️ PARTIAL SETUP - Some features may be limited")
            if not verification_results['sample_data']:
                print("   • Sample data missing. Ensure bundle extraction succeeded.")
            if not verification_results['simulation_imports']:
                print("   • Verify that src/ is present and on sys.path.")
        else:
            print("
❌ SETUP ISSUES - Multiple problems detected")
            print("   • Restart runtime and re-run cells in order")
            print("   • Confirm bundle download or upload steps completed")

        return verification_results

    verification_results = verify_complete_setup()


## 🎯 Step 7: Quick Test Simulation (Optional)

If all verification checks pass, you can run a quick test simulation.

In [ ]:
# Quick test simulation (only if setup is complete)
def run_quick_test():
    """Run a quick test simulation if setup is complete"""
    print("🧪 Quick Test Simulation")
    print("=" * 25)
    
    # Check if we can import simulation
    try:
        from av_simulation.data.repository import DataRepository
        from av_simulation.data.exporters import CSVExporter, JSONExporter
        
        # Create test repository
        is_colab = '/content' in os.getcwd()
        data_path = '/content/simulation_data' if is_colab else './simulation_data'
        
        repo = DataRepository(data_path)
        
        print("✅ AV Simulation imports successful!")
        print(f"✅ Data repository created at: {data_path}")
        print("🚗 Ready to run simulations!")
        
        # Test enhanced runner
        print(f"\n🧪 Testing enhanced runner...")
        memory_info = enhanced_runner.get_memory_usage()
        print(f"   📊 Current memory usage: {memory_info}")
        
        print("\n🎉 Test simulation setup successful!")
        print("\n📋 Ready for:")
        print("   ✅ Running simulations with enhanced_runner")
        print("   ✅ Data collection and analysis")
        print("   ✅ Screenshot capture and visualization")
        print("   ✅ Memory-optimized operations")
        
        return True
        
    except ImportError as e:
        print(f"❌ Import test failed: {e}")
        print("\n💡 This means:")
        print("   - Simulation files are not properly uploaded/cloned")
        print("   - Python path may not be configured correctly")
        print("   - Some dependencies may be missing")
        print("\n🔧 Solutions:")
        print("   1. Upload simulation files manually to the project directory")
        print("   2. Run the GitHub clone cell if repository is available")
        print("   3. Check the project structure verification above")
        
        return False
    
    except Exception as e:
        print(f"❌ Test failed: {e}")
        return False

# Run quick test if verification passed
if verification_results and sum(verification_results.values()) >= len(verification_results) * 0.7:
    test_success = run_quick_test()
else:
    print("⚠️ Skipping quick test due to setup issues")
    print("Please address the verification issues above first")

## 🔧 Troubleshooting Guide

### Common Issues and Solutions:

**🖥️ Display Issues:**
- **Problem**: `Display setup failed` or `pygame display error`
- **Solution**: Restart runtime and re-run cells 1-3 in order
- **Alternative**: Check if Xvfb is properly installed

**📦 Import Errors:**
- **Problem**: `Cannot import av_simulation` modules
- **Solution**: Upload simulation files manually or check GitHub clone
- **Check**: Verify project structure in Step 6

**💾 Memory Issues:**
- **Problem**: `High memory usage` warnings
- **Solution**: Use `enhanced_runner.clear_screenshots()` to free memory
- **Prevention**: Reduce `max_screenshots` parameter

**🌐 Network Issues:**
- **Problem**: GitHub clone fails or package installation fails
- **Solution**: Check internet connection and try manual upload
- **Alternative**: Use the basic project structure option

### Performance Optimization:

**For long simulations:**
- Increase `screenshot_interval` to reduce memory usage
- Use `max_screenshots=25` for memory-constrained environments
- Limit simulation `duration` to 5 minutes or less

**For better visualization:**
- Use `enhanced_runner.display_screenshots(max_images=9)` for more images
- Adjust `figsize=(20, 15)` for larger plots

### Getting Help:
- Check the verification results in Step 6
- Review error messages in previous cells
- Try restarting runtime if issues persist
- Consult the main project documentation

## 🎉 Setup Complete!

You've successfully set up the enhanced AV Simulation environment on Google Colab!

### ✅ What's Ready:
- **🖥️ Robust Virtual Display** - Virtual display with retry logic and error handling
- **📦 Comprehensive Dependencies** - All required packages with verification
- **📁 Dynamic Project Structure** - Automatic path detection and setup
- **💾 Memory Management** - Optimized screenshot storage and cleanup
- **🔍 Error Handling** - Comprehensive error detection and recovery
- **🧪 Enhanced Testing** - Detailed verification and diagnostics

### 🚀 Next Steps:
1. **Run Simulations** - Use the enhanced runner in other notebooks
2. **Collect Data** - Enable data collection for analysis
3. **Create Visualizations** - Generate plots and insights
4. **Analyze Results** - Explore simulation data and performance

### 📚 Available Notebooks:
- **[02_simulation_runner.ipynb](02_simulation_runner.ipynb)** - Run enhanced simulations
- **[03_data_analysis.ipynb](03_data_analysis.ipynb)** - Analyze simulation data
- **[04_visualization_examples.ipynb](04_visualization_examples.ipynb)** - Create visualizations
- **[05_advanced_analysis.ipynb](05_advanced_analysis.ipynb)** - Advanced analytics

---

**🎯 Enhanced Setup Complete - Happy Simulating! 🚗💨**